In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import statsmodels.formula.api as smf
%matplotlib inline

In [2]:
def transform_column_names(dataframe):
    dataframe.columns = [x.lower() for x in dataframe.columns]
    dataframe.columns = [x.replace(" ","") for x in dataframe.columns]
    return dataframe.columns

In [3]:
def group_and_count(dataframe,feature):
    counts = dataframe.groupby(feature)['totalcharges'].count()
    return counts

In [4]:
def tranform_data(counts,feature,measured):
    col1 = counts.index.tolist()
    s1 = pd.Series(col1)

    col2 = []
    for count in counts:
        col2.append(count)
    s2 = pd.Series(col2)

    df2 = pd.concat([s1,s2],axis=1)
    df2.columns = [feature,measured]
    return df2

In [5]:
def plot_index(dataframe,feature1,feature2):
    index = np.arange(len(dataframe))
    plt.figure(figsize=(20,7))
    bar_width = 0.9
    plt.bar(index,dataframe[feature2], bar_width)
    plt.xticks(index, dataframe[feature1],rotation=45)
    plt.legend()
    return plt.show()

In [6]:
def pretty_print_linear(coefs, names = None, sort = False):
    if names == None:
        names = ["X%s" % x for x in range(len(coefs))]
    lst = zip(coefs, names)
    if sort:
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))
    return " + ".join("%s * %s" % (round(coef, 3), name)
                                   for coef, name in lst)

In [7]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',300)

In [8]:
df = pd.read_csv("Hospital_Inpatient_Discharges__SPARCS_De-Identified___2011.csv", low_memory=False)

In [9]:
transform_column_names(df)

Index([u'healthservicearea', u'hospitalcounty', u'operatingcertificatenumber', u'facilityid', u'facilityname', u'agegroup', u'zipcode-3digits', u'gender', u'race', u'ethnicity', u'lengthofstay', u'admitdayofweek', u'typeofadmission', u'patientdisposition', u'dischargeyear', u'dischargedayofweek', u'ccsdiagnosiscode', u'ccsdiagnosisdescription', u'ccsprocedurecode', u'ccsproceduredescription', u'aprdrgcode', u'aprdrgdescription', u'aprmdccode', u'aprmdcdescription', u'aprseverityofillnesscode', u'aprseverityofillnessdescription', u'aprriskofmortality', u'aprmedicalsurgicaldescription', u'sourceofpayment1', u'sourceofpayment2', u'sourceofpayment3', u'attendingproviderlicensenumber', u'operatingproviderlicensenumber', u'otherproviderlicensenumber', u'birthweight', u'abortioneditindicator', u'emergencydepartmentindicator', u'totalcharges', u'totalcosts'], dtype='object')

In [10]:
# remove empty columns of other payment source (2,3) before dropping NAN values
del df['sourceofpayment2']
del df['sourceofpayment3']

In [11]:
df['lengthofstay'] = df.lengthofstay.str.replace('+', '').astype(float)

In [12]:
df['ccsdiagnosiscode'] = df['ccsdiagnosiscode'].astype(float)

In [13]:
df['totalcharges'] = df['totalcharges'].str.replace(r'$','').astype(float)
df['totalcosts'] = df['totalcosts'].str.replace(r'$','').astype(float)

In [14]:
CS = df[df['ccsproceduredescription'] == 'CESAREAN SECTION']

In [109]:
CS.head()

,healthservicearea,hospitalcounty,operatingcertificatenumber,facilityid,facilityname,agegroup,zipcode-3digits,gender,race,ethnicity,lengthofstay,admitdayofweek,typeofadmission,patientdisposition,dischargeyear,dischargedayofweek,ccsdiagnosiscode,ccsdiagnosisdescription,ccsprocedurecode,ccsproceduredescription,aprdrgcode,aprdrgdescription,aprmdccode,aprmdcdescription,aprseverityofillnesscode,aprseverityofillnessdescription,aprriskofmortality,aprmedicalsurgicaldescription,sourceofpayment1,attendingproviderlicensenumber,operatingproviderlicensenumber,otherproviderlicensenumber,birthweight,abortioneditindicator,emergencydepartmentindicator,totalcharges,totalcosts
0,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,120,F,White,Not Span/Hispanic,8,SAT,Emergency,Home or Self Care,2011,SUN,190,FETAL DISTRESS,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",3,Major,Minor,Surgical,Insurance Company,114221,171241,114221,0,N,Y,25584.21,11622.46
1,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,18 to 29,128,F,White,Not Span/Hispanic,4,SAT,Emergency,Home or Self Care,2011,WED,182,PREGNANCY HEMORRHAG,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",2,Moderate,Minor,Surgical,Insurance Company,196010,254742,254742,0,N,Y,41648.20,11220.69
2,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,120,F,White,Not Span/Hispanic,17,WED,Emergency,Home w/ Home Health Services,2011,SAT,195,OTH COMP BIRTH/PUERPRM,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",3,Major,Major,Surgical,Insurance Company,149551,149551,149551,0,N,Y,128973.98,48055.58
3,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,122,F,Black/African American,Not Span/Hispanic,11,FRI,Emergency,Home or Self Care,2011,TUE,182,PREGNANCY HEMORRHAG,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",3,Major,Moderate,Surgical,Insurance Company,148296,148296,148296,0,N,Y,76803.96,27072.54
4,Capital/Adiron,Albany,101000,1,Albany Medical Center Hospital,30 to 49,120,F,White,Not Span/Hispanic,3,SUN,Elective,Home or Self Care,2011,WED,181,OTHR PREGNANCY COMPL,134,CESAREAN SECTION,540,CESAREAN DELIVERY,14,"Pregnancy, Childbirth and the Puerperium",2,Moderate,Minor,Surgical,Medicaid,188988,188988,188988,0,N,N,18870.56,8738.00


In [84]:
CS = CS.reset_index(drop=True)

In [16]:
CS = CS.dropna()

In [85]:
CS.to_csv("NY_CesareanSection_2011.csv")

In [17]:
features = ['hospitalcounty','facilityname','agegroup','typeofadmission','ccsdiagnosisdescription',\
            'aprdrgdescription','aprmdcdescription','aprseverityofillnessdescription','aprriskofmortality',\
            'attendingproviderlicensenumber','sourceofpayment1']

In [95]:
features = ['facilityname','agegroup','typeofadmission','ccsdiagnosisdescription',\
            'aprdrgdescription','aprmdcdescription','aprseverityofillnessdescription','aprriskofmortality',\
            'attendingproviderlicensenumber','sourceofpayment1']

In [96]:
model_df = CS['totalcharges']
model_df.shape

(13552,)

In [97]:
model_df = pd.concat([model_df,CS['lengthofstay']],axis=1)
model_df.shape

(13552, 2)

In [98]:
for feature in features:
        model_df = pd.concat([model_df,pd.get_dummies(CS[feature])],axis=1)
        concatinated = list(pd.get_dummies(CS[feature]))
        redundant = concatinated.pop()
        model_df.drop(redundant, axis=1, inplace=True)
model_df.shape

(13552, 934)

In [99]:
from sklearn import preprocessing
### Model input:

#print model_df['totalcharges'].head()
#print X_values.head()

X_values = model_df.copy() # lots of junk - only length of stay matters
X_values.drop(['totalcharges','lengthofstay'],axis=1,inplace=True)
X_values_scaled = preprocessing.scale(X_values, copy=False)

In [100]:
y_values = model_df['totalcharges']/model_df['lengthofstay']


In [101]:
y_values.isnull().any().any()

False

In [54]:
y_values

0         3198.026250
1        10412.050000
3         7586.704706
4         6982.178182
6         6290.186667
8         5499.343333
9         4930.477500
10        3004.184667
11        4458.893333
12        4756.247500
13        2659.750000
14        4743.270000
15        7720.965000
16        4861.306667
17        4909.993333
18        4941.380000
19        4482.114000
20        4747.320000
21        6587.820000
22        7710.150000
23        4300.485714
24        5320.770000
26        7089.920000
27        5862.305000
28        4200.143333
29        4702.835000
30        2948.425000
31        6175.250000
32        2838.796667
33        4295.901429
34        3937.436667
35        2806.257500
36        4571.546667
37        5779.102500
38        2693.892500
39        3989.665000
40        5336.674000
41        7259.183333
42        2708.316667
43        4572.157500
44        8219.696667
45        3794.436667
46        5421.110000
47        3533.814444
48        4304.893333
49        

In [102]:
names = list(X_values)
print len(names)
#names

932


In [103]:
X_values.isnull().any().any()  # check if any values are NaN

False

In [104]:
y_values.isnull().any().any()

False

### Errors for Cross-Validation

In [26]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

#### Lasso Regression

In [32]:
from sklearn.cross_validation import cross_val_score, KFold,ShuffleSplit
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#X = scaler.fit_transform(X_values)
X = X_values_scaled
y = y_values

In [34]:
lasso = Lasso(alpha=.3)

lasso_scores = []
lasso_MSEs = []

for i in range(X.shape[1]):
     lasso_score = cross_val_score(lasso, X[:, i:i+1], y, scoring="r2",cv=ShuffleSplit(len(X), 3, .3))
     lasso_MSE =   cross_val_score(lasso, X[:, i:i+1], y, scoring="mean_squared_error",cv=ShuffleSplit(len(X), 3, .3))
     lasso_scores.append((round(np.mean(lasso_score), 3), names[i]))
     lasso_MSEs.append((round(np.mean(lasso_MSE),3),names[i]))
print sorted(lasso_scores, reverse=True)
print
print sorted(lasso_MSEs, reverse=True)

[(0.088, 'Manhattan'), (0.084, 'NYU Hospitals Center'), (0.053, 'Rockland'), (0.052, 'Good Samaritan Hospital of Suffern'), (0.045, 'Queens'), (0.045, 'Bronx'), (0.04, 'Flushing Hospital Medical Center'), (0.038, 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'), (0.034, 'Montefiore Medical Center - North Division'), (0.033, 'University Hospital of Brooklyn'), (0.033, 'Emergency'), (0.027, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.025, 'Vassar Brothers Medical Center'), (0.025, 'Mount Sinai Hospital'), (0.02, '18 to 29'), (0.019, 'South Nassau Communities Hospital'), (0.019, 'Nassau'), (0.017, 'Medicaid'), (0.017, 'Insurance Company'), (0.017, 'Aurelia Osborn Fox Memorial Hospital'), (0.016, '30 to 49'), (0.015, 'Dutchess'), (0.014, 'Otsego'), (0.014, 'Orange'), (0.014, 'Minor'), (0.013, 'Rensselaer'), (0.013, 'Major'), (0.012, 'Samaritan Hospital'), (0.012, 'Maimonides Medical Center'), (0.011, 'Jefferson'), (0.01, 'Samaritan Medical Center'), (0.

In [35]:
lasso.fit(X, y)
print "Lasso model:", pretty_print_linear(lasso.coef_,names,sort=True)

Lasso model: 728.445 * Manhattan + 603.318 * Rockland + 507.473 * Maimonides Medical Center + -419.03 * Bronx-Lebanon Hospital Center - Concourse Division + 414.946 * Bronx + 386.174 * 222398.0 + 305.803 * NYU Hospitals Center + 301.804 * Montefiore Medical Center - North Division + 297.68 * Vassar Brothers Medical Center + 272.405 * Orange + 231.532 * Dutchess + 231.034 * Major + 214.764 * Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div + -183.203 * Flushing Hospital Medical Center + -179.101 * Jefferson + -174.409 * Otsego + 169.108 * Extreme + -149.765 * Erie + -147.064 * Montgomery + -139.344 * Insurance Company + -139.002 * Chemung + -132.79 * Blue Cross + -131.935 * University Hospital of Brooklyn + -125.141 * Cattaraugus + -120.189 * Rensselaer + -119.125 * Samaritan Hospital + -118.168 * Northern Westchester Hospital + -118.132 * Nassau + 116.427 * Orange Regional Medical Center-Goshen Campus + 98.997 * 212628.0 + 93.768 * Other Non-Federal Program + -91.28

/usr/local/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:444: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [79]:
from collections import defaultdict
Lasso_scores = defaultdict(list)
Lasso_algh_score = []

for train_idx, test_idx in ShuffleSplit(len(X), 3, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = y[train_idx], y[test_idx]
    
    r = lasso.fit(X_train, Y_train)
    acc = r2_score(Y_test, lasso.predict(X_test))
    Lasso_algh_score.append(acc)
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, lasso.predict(X_t))
        Lasso_scores[names[i]].append((acc-shuff_acc)/acc)
LRsquare = np.mean(Lasso_algh_score)
print 'LRsquare =', LRsquare       
print "Features sorted by their score:"
print sorted([(round(np.mean(score), 4), feat) for feat, score in Lasso_scores.items()], reverse=True)

LRsquare = 0.462885648873
Features sorted by their score:
[(0.4717, 'Manhattan'), (0.3336, 'Rockland'), (0.2338, 'Maimonides Medical Center'), (0.1667, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.1659, 'Bronx'), (0.0952, 'Montefiore Medical Center - North Division'), (0.0879, 'Vassar Brothers Medical Center'), (0.0731, 'NYU Hospitals Center'), (0.0534, 'Dutchess'), (0.048, 'Otsego'), (0.046, 'Orange'), (0.0398, 'Montgomery'), (0.0304, 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'), (0.0297, 'Jefferson'), (0.0283, 'Flushing Hospital Medical Center'), (0.023, 'Brookdale Hospital Medical Center'), (0.0223, 'Blue Cross'), (0.0196, 'Insurance Company'), (0.0163, 'Extreme'), (0.0162, 'Chemung'), (0.0147, 'Medicaid'), (0.0137, 'Nassau'), (0.0134, 'Cattaraugus'), (0.0132, 'Samaritan Hospital'), (0.0125, 'Rensselaer'), (0.0122, 'Northern Westchester Hospital'), (0.0107, 'Orange Regional Medical Center-Goshen Campus'), (0.0097, 'University Hospital of Brook

In [88]:
import statsmodels.api as sm

model = sm.OLS(y,X)
results = model.fit()
print results.summary()
#statsmodels.regression.linear_model.OLS.fit_regularized
#model = regression.linear_model.OLS.fit_regularized.fit_regularized()
#model.fit(y,X)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                 -0.028
Method:                 Least Squares   F-statistic:                    0.5900
Date:                Tue, 23 Jun 2015   Prob (F-statistic):               1.00
Time:                        13:53:39   Log-Likelihood:            -1.3623e+05
No. Observations:               13552   AIC:                         2.742e+05
Df Residuals:                   12660   BIC:                         2.810e+05
Df Model:                         892                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1          4.088e+15   3.35e+15      1.218      0.2

In [92]:
print(X_values).head()

     Albany  Allegheny     Bronx    Broome  Cattaraugus    Cayuga  \
0  4.060351  -0.019212 -0.339828 -0.021046    -0.077543 -0.019212   
1  4.060351  -0.019212 -0.339828 -0.021046    -0.077543 -0.019212   
3  4.060351  -0.019212 -0.339828 -0.021046    -0.077543 -0.019212   
4  4.060351  -0.019212 -0.339828 -0.021046    -0.077543 -0.019212   
6  4.060351  -0.019212 -0.339828 -0.021046    -0.077543 -0.019212   

   Chautatuqua   Chemung  Columbia  Dutchess      Erie  Franklin    Fulton  \
0    -0.012149 -0.072056 -0.049407 -0.260629 -0.101799  -0.04468 -0.105078   
1    -0.012149 -0.072056 -0.049407 -0.260629 -0.101799  -0.04468 -0.105078   
3    -0.012149 -0.072056 -0.049407 -0.260629 -0.101799  -0.04468 -0.105078   
4    -0.012149 -0.072056 -0.049407 -0.260629 -0.101799  -0.04468 -0.105078   
6    -0.012149 -0.072056 -0.049407 -0.260629 -0.101799  -0.04468 -0.105078   

    Genesee  Jefferson     Kings  Madison  Manhattan    Monroe  Montgomery  \
0 -0.109649  -0.109994 -0.603379 -0.00

In [105]:
X_values['Ones'] = 1

In [108]:
model = sm.OLS(y,X_values)
results = model.fit_regularized(alpha=0.3)
print results.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.575
Method:                 Least Squares   F-statistic:                     21.51
Date:                Tue, 23 Jun 2015   Prob (F-statistic):               0.00
Time:                        15:54:29   Log-Likelihood:            -1.1823e+05
No. Observations:               13552   AIC:                         2.382e+05
Df Residuals:                   12659   BIC:                         2.450e+05
Df Model:                         892                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------

#### Ridge Regression

In [36]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=10)

In [38]:
ridge_scores = []
for i in range(X.shape[1]):
     ridge_score = cross_val_score(lasso, X[:, i:i+1], y, scoring="r2",cv=ShuffleSplit(len(X), 3, .3))
     ridge_scores.append((round(np.mean(ridge_score), 3), names[i]))
print sorted(ridge_scores, reverse=True)

[(0.102, 'Manhattan'), (0.082, 'NYU Hospitals Center'), (0.055, 'Queens'), (0.047, 'Good Samaritan Hospital of Suffern'), (0.046, 'Montefiore Medical Center - North Division'), (0.041, 'Rockland'), (0.04, 'Emergency'), (0.036, 'Bronx'), (0.032, 'Flushing Hospital Medical Center'), (0.031, 'University Hospital of Brooklyn'), (0.031, 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'), (0.027, 'Mount Sinai Hospital'), (0.025, 'Orange'), (0.022, 'South Nassau Communities Hospital'), (0.02, 'Vassar Brothers Medical Center'), (0.02, 'Nassau'), (0.02, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.019, 'Medicaid'), (0.019, '18 to 29'), (0.018, 'Otsego'), (0.018, 'Aurelia Osborn Fox Memorial Hospital'), (0.018, '30 to 49'), (0.015, 'Insurance Company'), (0.015, 'Dutchess'), (0.014, 'Rensselaer'), (0.013, 'Maimonides Medical Center'), (0.012, 'Orange Regional Medical Center-Goshen Campus'), (0.012, 'Jefferson'), (0.011, 'Samaritan Hospital'), (0.011, 'Minor'), (0

In [39]:
ridge.fit(X,y)
print "Ridge model:", pretty_print_linear(ridge.coef_)
print

Ridge model: -8.205 * X0 + -20.335 * X1 + 143.413 * X2 + -21.38 * X3 + -67.038 * X4 + 4.393 * X5 + -13.285 * X6 + -67.685 * X7 + -32.802 * X8 + 109.793 * X9 + -104.07 * X10 + -26.87 * X11 + 35.66 * X12 + 12.491 * X13 + -130.659 * X14 + -38.65 * X15 + -0.244 * X16 + 225.371 * X17 + -48.076 * X18 + -90.499 * X19 + -99.055 * X20 + -33.549 * X21 + -33.501 * X22 + 111.37 * X23 + -57.135 * X24 + -146.899 * X25 + -182.298 * X26 + 174.583 * X27 + -5.663 * X28 + -9.381 * X29 + -42.558 * X30 + 20.612 * X31 + -32.432 * X32 + -3.545 * X33 + -8.205 * X34 + -26.87 * X35 + -67.685 * X36 + 4.393 * X37 + -57.135 * X38 + 60.475 * X39 + -202.558 * X40 + 61.283 * X41 + -166.394 * X42 + -13.285 * X43 + 20.612 * X44 + -32.802 * X45 + -42.558 * X46 + -5.663 * X47 + -177.447 * X48 + -3.545 * X49 + 174.717 * X50 + -9.668 * X51 + 0.084 * X52 + -32.432 * X53 + 13.368 * X54 + -10.661 * X55 + 162.75 * X56 + -20.335 * X57 + 40.154 * X58 + -42.615 * X59 + 116.181 * X60 + 38.55 * X61 + 234.708 * X62 + 42.583 * X63 + 

In [80]:
from collections import defaultdict
Ridge_scores = defaultdict(list)
Ridge_algh_score = []

for train_idx, test_idx in ShuffleSplit(len(X), 3, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = y[train_idx], y[test_idx]
    
    r = ridge.fit(X_train, Y_train)
    acc = r2_score(Y_test, ridge.predict(X_test))
    Ridge_algh_score.append(acc)
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, ridge.predict(X_t))
        Ridge_scores[names[i]].append((acc-shuff_acc)/acc)
Ridge_R2 = np.mean(Ridge_algh_score)
print 'Ridge_R2 =', Ridge_R2               
print "Features sorted by their score:"
print sorted([(round(np.mean(score), 4), feat) for feat, score in Ridge_scores.items()], reverse=True)

Ridge_R2 = 0.476973033742
Features sorted by their score:
[(0.0532, 'NYU Hospitals Center'), (0.0495, 'Montefiore Medical Center - North Division'), (0.0448, 'Manhattan'), (0.0323, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.029, 'Rockland'), (0.0272, 'Samaritan Hospital'), (0.0266, 'Rensselaer'), (0.0264, 'Good Samaritan Hospital of Suffern'), (0.0252, 'Insurance Company'), (0.0228, 'Flushing Hospital Medical Center'), (0.022, 'Maimonides Medical Center'), (0.0192, 'Bronx'), (0.0188, 'Queens'), (0.0183, 'Sisters of Charity Hospital'), (0.0159, 'Orange'), (0.0158, 'Blue Cross'), (0.0151, 'Vassar Brothers Medical Center'), (0.0149, 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'), (0.0139, 'Brooklyn Hospital Center - Downtown Campus'), (0.0138, 'Samaritan Medical Center'), (0.0135, 'University Hospital of Brooklyn'), (0.0128, 'Extreme'), (0.0128, 248441.0), (0.0127, 253610.0), (0.0126, 'Orange Regional Medical Center-Goshen Campus'), (0.0126, 'Erie')

#### Random Forest Regressor

In [40]:
#### RF do not require scaling
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [41]:
scores = []
for i in range(X.shape[1]):
     score = cross_val_score(rf, X[:, i:i+1], y, scoring="r2",cv=ShuffleSplit(len(X), 3, .3))
     scores.append((round(np.mean(score), 3), names[i]))
print sorted(scores, reverse=True)

[(0.107, 'Manhattan'), (0.088, 'NYU Hospitals Center'), (0.051, 'Good Samaritan Hospital of Suffern'), (0.05, 'Queens'), (0.048, 'Montefiore Medical Center - North Division'), (0.048, 'Flushing Hospital Medical Center'), (0.043, 'Rockland'), (0.043, 'Bronx'), (0.034, 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'), (0.031, 'Emergency'), (0.029, 'University Hospital of Brooklyn'), (0.025, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.023, 'Mount Sinai Hospital'), (0.021, 'Dutchess'), (0.02, 'Vassar Brothers Medical Center'), (0.02, 'South Nassau Communities Hospital'), (0.019, 'Aurelia Osborn Fox Memorial Hospital'), (0.017, 'Nassau'), (0.017, 'Medicaid'), (0.017, '30 to 49'), (0.016, '18 to 29'), (0.015, 'Otsego'), (0.014, 'Orange'), (0.014, 'Insurance Company'), (0.013, 'Rensselaer'), (0.013, 'Minor'), (0.012, 'Nathan Littauer Hospital'), (0.012, 'Major'), (0.012, 'Maimonides Medical Center'), (0.011, 'Jamaica Hospital Medical Center'), (0.01, 'Sama

In [42]:
rf.fit(X, y)
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True)

Features sorted by their score:
[(0.1147, 'Manhattan'), (0.074, 'Vassar Brothers Medical Center'), (0.0721, 'Bronx'), (0.0664, 'Maimonides Medical Center'), (0.0659, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.0646, 212628.0), (0.0577, 'Rockland'), (0.0424, 'Orange'), (0.0155, 'Major'), (0.0145, 'Major'), (0.0136, 'Good Samaritan Hospital of Suffern'), (0.0136, 222398.0), (0.0125, 'Minor'), (0.0121, '30 to 49'), (0.0115, 'PREVIOUS C-SECTION'), (0.0114, 'Minor'), (0.0109, 'OTH COMP BIRTH/PUERPRM'), (0.01, 'Insurance Company'), (0.01, '18 to 29'), (0.0094, 'Albany Medical Center Hospital'), (0.0092, 'Blue Cross'), (0.0083, 'PREGNANCY HEMORRHAG'), (0.008, 'CHAMPUS'), (0.0078, 'Mount Sinai Hospital'), (0.0071, 'PREGNANCY HYPERTENSN'), (0.0069, 'Brookdale Hospital Medical Center'), (0.0067, 'Elective'), (0.0066, 'Medicaid'), (0.0066, 'Extreme'), (0.0064, 'Albany'), (0.0063, 'FETAL DISTRESS'), (0.0059, 'MAL-POSITION/PRESNTATN'), (0.0053, 'NYU Hospitals Center'), (0.0049, 'Extre

In [60]:
y_values

0         3198.026250
1        10412.050000
3         7586.704706
4         6982.178182
6         6290.186667
8         5499.343333
9         4930.477500
10        3004.184667
11        4458.893333
12        4756.247500
13        2659.750000
14        4743.270000
15        7720.965000
16        4861.306667
17        4909.993333
18        4941.380000
19        4482.114000
20        4747.320000
21        6587.820000
22        7710.150000
23        4300.485714
24        5320.770000
26        7089.920000
27        5862.305000
28        4200.143333
29        4702.835000
30        2948.425000
31        6175.250000
32        2838.796667
33        4295.901429
34        3937.436667
35        2806.257500
36        4571.546667
37        5779.102500
38        2693.892500
39        3989.665000
40        5336.674000
41        7259.183333
42        2708.316667
43        4572.157500
44        8219.696667
45        3794.436667
46        5421.110000
47        3533.814444
48        4304.893333
49        

In [81]:
from collections import defaultdict
T_scores = defaultdict(list)
RF_algh_score = []

for train_idx, test_idx in ShuffleSplit(len(X), 3, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = y[train_idx], y[test_idx]
    
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    RF_algh_score.append(acc)
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        T_scores[names[i]].append((acc-shuff_acc)/acc)
RF_R2 = np.mean(RF_algh_score)
print 'RF_R2 =', RF_R2          
print "Features sorted by their score:"
print sorted([(round(np.mean(score), 4), feat) for feat, score in T_scores.items()], reverse=True)


RF_R2 = 0.423955985208
Features sorted by their score:
[(0.93, 'Manhattan'), (0.6694, 'Bronx'), (0.4356, 'Maimonides Medical Center'), (0.3197, 'Vassar Brothers Medical Center'), (0.1873, 'Bronx-Lebanon Hospital Center - Concourse Division'), (0.1855, 'Rockland'), (0.1668, 'Orange'), (0.1366, 212628.0), (0.0784, 'Good Samaritan Hospital of Suffern'), (0.0589, 'Major'), (0.0279, 'Brookdale Hospital Medical Center'), (0.0172, 'NYU Hospitals Center'), (0.017, 'Elective'), (0.016, 'SUNY Downstate Medical Center at LICH'), (0.0128, 'Otsego'), (0.0128, 'Albany Medical Center Hospital'), (0.0116, 'Minor'), (0.0093, 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'), (0.0087, 'FETAL DISTRESS'), (0.0087, 'Albany'), (0.0053, 'CHAMPUS'), (0.0048, 'Samaritan Hospital'), (0.0048, 'Montefiore Medical Center - North Division'), (0.0039, 'Extreme'), (0.0031, 'Emergency'), (0.0028, 'PREVIOUS C-SECTION'), (0.0028, 'Jamaica Hospital Medical Center'), (0.0027, 'Aurelia Osborn Fox Memo